In [43]:
# import the relevant libraries

import datetime
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime
from pptx import Presentation
from pptx.util import Inches
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN
from datetime import date
from pptx.dml.color import RGBColor

import os
import sys
from scipy import stats

# Import the required tkinter Libraries
import tkinter as tk
from tkinter import *
from tkinter import ttk, filedialog
#from tkinter.filedialog import askopenfile
from tkinter import messagebox
from threading import Thread
import zipfile

##Image libraries
import plotly.io as pio
import tempfile
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [61]:
def main(csv_file_list, stack_info_fn, output_dir, Unit_name):
    image_folder_name = '{}/images'.format(output_dir)
    if not os.path.exists(image_folder_name):
        os.mkdir(image_folder_name)
    
    if Unit_name == 'U05':
        template_ppt_fn = 'ckb_template_U05'
    elif Unit_name == 'U0002':
        template_ppt_fn = 'ckb_template_U0002'
    elif Unit_name == 'SPR0001':
        template_ppt_fn = 'ntpc_template'
    ppt_fn = datetime.now().strftime("%Y-%B-%d_%H%M"+"hrs_IST_") + Unit_name +"_update"
    print("Output Rreport name will be:\n " + ppt_fn)
    ## giving user input for Stack position and Stackid    
    Stack_information = pd.read_excel(stack_info_fn, sheet_name = Unit_name, header=0)
    table_data = Stack_information.values.tolist()
    
    print(csv_file_list)
    # path = "./extracted_files/"
    # image_folder_name = './images'
    sys_data = pd.concat([pd.read_csv(f, error_bad_lines=False) for f in csv_file_list ], ignore_index=True)
    
    #Calculating run hrs by considering timegap more than 10 sec as an interruption.

    # converting datetime from string to datetime format
    sys_data['Time'] = pd.to_datetime(sys_data['Time'],dayfirst=True)
    ## initialising run_hrs and a duration list
    run_hrs = 0
    duration_hr = [0.000]

    ## definig the run hrs completed by defining delta(s)    
    for i in range (1,len(sys_data)):
        s = (sys_data['Time'][i] - sys_data['Time'][i-1]).seconds
        if s <= 300 :
            run_hrs = run_hrs + (s/3600)
            duration_hr.append(run_hrs)
        elif s > 300:
            run_hrs = run_hrs + (10/3600)
            duration_hr.append(run_hrs)
    #adding a column for duration in dataframe
    sys_data['duration_hr'] = duration_hr        
    # sys_data[['Time','duration_hr']].head()
    # Output the total run duration in round figure in hrs (excluding interruptions)

    total_duration_hrs = round(max(sys_data['duration_hr']),0)
    print("Total Run Duration is "+ '"'+str(int(total_duration_hrs))+" hours"+'" (excluding interruptions)')
    Start_time = str(min(sys_data['Time']))
    End_time = str(max(sys_data['Time']))

    print("From : "+Start_time)
    print("To   : "+End_time)

    # replace the the string '-NA-' in data with blank
    sys_data = sys_data.replace('-NA-',np.nan)

    #remove rows that have any blank values
    # sys_data = sys_data.dropna(axis = 0)
    
    ## pre-processing the data ##
    ##remove the blank cols from data
    sys_data_empty = []
    for i in sys_data.columns:
        if len(sys_data[i].value_counts()) == 0:
            sys_data_empty.append(i)
            sys_data = sys_data.drop(i,axis = 1)        
    # print("The following fields are completely blank in the data, hence are removed & stored separately:", sys_data_empty)

    #convert the required fields from object to numeric datatype
    obj_cols = sys_data.loc[:, ~sys_data.columns.isin(['Time', 'MP State', 'MS State','Train 1 State', 'Train 2 State', 'PMP 102'])].columns
    sys_data[obj_cols] = sys_data[obj_cols].apply(pd.to_numeric, errors='coerce', axis=1)
    #make all the col names in uppercase
    sys_data.columns = sys_data.columns.str.upper()
    #rename TIME col as Date
    sys_data.rename(columns= {'TIME':'Date'},inplace = True)
    
    Vcell_data = sys_data
    # make a dict containing the positions of the last of the 56 cells
    positions = {'U1': 4, 'U2': 8, 'L2': 12, 'L1': 16}
    # loop through this dict
    for position, cell in positions.items():
        # drop the unused cells (57-64)
        Vcell_data.drop(columns=['C{}-{}'.format(cell, i) for i in range(9,17)], inplace=True)
        # get the known start and end cells (1-56)
        start = 'C{}-1'.format(str(cell-3))
        end = 'C{}-8'.format(str(cell))
        # define the active condition for the columns
        active_condition = (Vcell_data['MP STATE'] == 'STACK PWR') & (Vcell_data['MS STATE'] == 'ARMED')
        # take a temp df where the active condition is satisfied and get the time and current
        temp = Vcell_data.loc[active_condition, ['Date', 'STACK CURRENT']]
        # calculate the median of the cell voltages (some times voltages are strings so need to convert to floats)
        temp['{} median'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').median(axis=1, numeric_only=True)
        # calculate the mean of the cell voltages
        temp['{} mean'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').mean(axis=1, numeric_only=True)
        # calculate the min of the cell voltages
        temp['{} min'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').min(axis=1, numeric_only=True)
        # calculate the max of the cell voltages
        temp['{} max'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').max(axis=1, numeric_only=True)
        # calculate the standard deviation of the cell voltages
        temp['{} std'.format(position)] = Vcell_data.loc[active_condition,start:end].astype('float64').std(axis=1, numeric_only=True)
    #     print(temp.head())
    #     make a column that indicates if a stack is present in the current position based on
    #     if the median voltage is greater than 1.4 when the current is > 9
        temp['{} present'.format(position)] = (temp['{} median'.format(position)] > 1.4) | ((temp['{} median'.format(position)] > 1.4) & (temp['STACK CURRENT'] >= 9))
    #     print(temp.head())

        # update the stack present column to be nan whenever the current is less than 9
        temp.loc[((temp['STACK CURRENT'] < 9) & (temp['{} present'.format(position)] == False)), '{} present'.format(position)] = np.nan
    #     temp.to_csv(position + '.csv')

        # drop the time and stack current columns before joining the temp df with the original df
        temp.drop(columns=['Date', 'STACK CURRENT'], inplace=True)
        # join the temp df where True with the original df
        Vcell_data = pd.concat([Vcell_data, temp], axis = 1)
        
    List_C1_1_C4_16 = [
        'C1-1', 'C1-2', 'C1-3', 'C1-4', 'C1-5', 'C1-6', 'C1-7', 'C1-8', 'C1-9', 'C1-10', 'C1-11', 'C1-12', 'C1-13', 'C1-14', 'C1-15', 'C1-16', 'C2-1', 'C2-2', 'C2-3', 'C2-4', 'C2-5', 'C2-6', 'C2-7', 'C2-8', 'C2-9', 'C2-10', 'C2-11', 'C2-12', 'C2-13', 'C2-14', 'C2-15', 'C2-16', 'C3-1', 'C3-2', 'C3-3', 'C3-4', 'C3-5', 'C3-6', 'C3-7', 'C3-8', 'C3-9', 'C3-10', 'C3-11', 'C3-12', 'C3-13', 'C3-14', 'C3-15', 'C3-16', 'C4-1', 'C4-2', 'C4-3', 'C4-4', 'C4-5', 'C4-6', 'C4-7', 'C4-8'
    # #                    ,'C4-9', 'C4-10', 'C4-11', 'C4-12', 'C4-13', 'C4-14', 'C4-15', 'C4-16'
                      ]


    List_C5_1_C8_16 = [
    'C5-1', 'C5-2', 'C5-3', 'C5-4', 'C5-5', 'C5-6', 'C5-7', 'C5-8', 'C5-9', 'C5-10', 'C5-11', 'C5-12', 'C5-13', 'C5-14', 'C5-15', 'C5-16', 'C6-1', 'C6-2', 'C6-3', 'C6-4', 'C6-5', 'C6-6', 'C6-7', 'C6-8', 'C6-9', 'C6-10', 'C6-11', 'C6-12', 'C6-13', 'C6-14', 'C6-15', 'C6-16', 'C7-1', 'C7-2', 'C7-3', 'C7-4', 'C7-5', 'C7-6', 'C7-7', 'C7-8', 'C7-9', 'C7-10', 'C7-11', 'C7-12', 'C7-13', 'C7-14', 'C7-15', 'C7-16'       
                       ,'C8-1', 'C8-2', 'C8-3', 'C8-4', 'C8-5', 'C8-6', 'C8-7', 'C8-8'
    #                    , 'C8-9', 'C8-10', 'C8-11', 'C8-12', 'C8-13', 'C8-14', 'C8-15', 'C8-16'
                      ]

    List_C9_1_C12_16 = [
        'C9-1', 'C9-2', 'C9-3', 'C9-4', 'C9-5', 'C9-6', 'C9-7', 'C9-8', 'C9-9', 'C9-10', 'C9-11', 'C9-12', 'C9-13', 'C9-14', 'C9-15', 'C9-16', 'C10-1', 'C10-2', 'C10-3', 'C10-4', 'C10-5', 'C10-6', 'C10-7', 'C10-8', 'C10-9', 'C10-10', 'C10-11', 'C10-12', 'C10-13', 'C10-14', 'C10-15', 'C10-16', 'C11-1', 'C11-2', 'C11-3', 'C11-4', 'C11-5', 'C11-6', 'C11-7', 'C11-8', 'C11-9', 'C11-10', 'C11-11', 'C11-12', 'C11-13', 'C11-14', 'C11-15', 'C11-16', 'C12-1', 'C12-2', 'C12-3', 'C12-4', 'C12-5', 'C12-6', 'C12-7', 'C12-8'
    #                    , 'C12-9', 'C12-10', 'C12-11', 'C12-12', 'C12-13', 'C12-14', 'C12-15', 'C12-16'
                      ]

    List_C13_1_C16_16 = [
        'C13-1', 'C13-2', 'C13-3', 'C13-4', 'C13-5', 'C13-6', 'C13-7', 'C13-8', 'C13-9', 'C13-10', 'C13-11', 'C13-12', 'C13-13', 'C13-14', 'C13-15', 'C13-16', 'C14-1', 'C14-2', 'C14-3', 'C14-4', 'C14-5', 'C14-6', 'C14-7', 'C14-8', 'C14-9', 'C14-10', 'C14-11', 'C14-12', 'C14-13', 'C14-14', 'C14-15', 'C14-16', 'C15-1', 'C15-2', 'C15-3', 'C15-4', 'C15-5', 'C15-6', 'C15-7', 'C15-8', 'C15-9', 'C15-10', 'C15-11', 'C15-12', 'C15-13', 'C15-14', 'C15-15', 'C15-16', 'C16-1', 'C16-2', 'C16-3', 'C16-4', 'C16-5', 'C16-6', 'C16-7', 'C16-8'
    #                     , 'C16-9', 'C16-10', 'C16-11', 'C16-12', 'C16-13', 'C16-14', 'C16-15', 'C16-16'
                       ]
    ## making all stack cells cols value to nan where stack present column is false or nan
    cell_range = {'U1':List_C1_1_C4_16, 'U2':List_C5_1_C8_16, 'L2':List_C9_1_C12_16, 'L1':List_C13_1_C16_16}
    for position , cells in cell_range.items():
        Vcell_data.loc[((Vcell_data['{} present'.format(position)].isna()) | (Vcell_data['{} present'.format(position)] == False)), cells] = np.nan

    Vcell_data['Avg_V_C1_1-C4_16'] = (Vcell_data[List_C1_1_C4_16].sum(axis=1))/len(List_C1_1_C4_16)
    Vcell_data['Max_V_C1_1-C4_16'] = Vcell_data[List_C1_1_C4_16].max(axis=1) 
    Vcell_data['Min_V_C1_1-C4_16'] = Vcell_data[List_C1_1_C4_16].min(axis=1)

    Vcell_data['Avg_V_C5_1-C8_16'] = (Vcell_data[List_C5_1_C8_16].sum(axis=1))/len(List_C5_1_C8_16)
    Vcell_data['Max_V_C5_1-C8_16'] = Vcell_data[List_C5_1_C8_16].max(axis=1)
    Vcell_data['Min_V_C5_1-C8_16'] = Vcell_data[List_C5_1_C8_16].min(axis=1)

    Vcell_data['Avg_V_C9_1-C12_16'] = (Vcell_data[List_C9_1_C12_16].sum(axis=1))/len(List_C9_1_C12_16)
    Vcell_data['Max_V_C9_1-C12_16'] = Vcell_data[List_C9_1_C12_16].max(axis=1)
    Vcell_data['Min_V_C9_1-C12_16'] = Vcell_data[List_C9_1_C12_16].min(axis=1)   

    Vcell_data['Avg_V_C13_1-C16_16'] = (Vcell_data[List_C13_1_C16_16].sum(axis=1))/len(List_C13_1_C16_16)
    Vcell_data['Max_V_C13_1-C16_16'] = Vcell_data[List_C13_1_C16_16].max(axis=1)
    Vcell_data['Min_V_C13_1-C16_16'] = Vcell_data[List_C13_1_C16_16].min(axis=1) 
    
    ## making cd, voltage charts., and ASR charts with dataframe created - Vcell_data
    Vcell_data['CURRENT DENSITY'] = Vcell_data['STACK CURRENT'] / 702

    #ASR in mohm-cm2
    Vcell_data['ASR_Stack1'] = (Vcell_data['Avg_V_C1_1-C4_16']/Vcell_data['CURRENT DENSITY'])*1000
    Vcell_data['ASR_Stack2'] = (Vcell_data['Avg_V_C5_1-C8_16']/Vcell_data['CURRENT DENSITY'])*1000
    Vcell_data['ASR_Stack3'] = (Vcell_data['Avg_V_C9_1-C12_16']/Vcell_data['CURRENT DENSITY'])*1000
    Vcell_data['ASR_Stack4'] = (Vcell_data['Avg_V_C13_1-C16_16']/Vcell_data['CURRENT DENSITY'])*1000

    Vcell_data['ASR(max)_Stack1'] = (Vcell_data['Max_V_C1_1-C4_16']/Vcell_data['CURRENT DENSITY'])*1000
    Vcell_data['ASR(max)_Stack2'] = (Vcell_data['Max_V_C5_1-C8_16']/Vcell_data['CURRENT DENSITY'])*1000 
    Vcell_data['ASR(max)_Stack3'] = (Vcell_data['Max_V_C9_1-C12_16']/Vcell_data['CURRENT DENSITY'])*1000
    Vcell_data['ASR(max)_Stack4'] = (Vcell_data['Max_V_C13_1-C16_16']/Vcell_data['CURRENT DENSITY'])*1000

    Current_Density = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['CURRENT DENSITY'],
                           mode = 'markers',
                           marker=dict(color='#2ca02c'),
                           showlegend = True,
    #                        mode = 'lines',
                           name = 'Current Density (A/c'+'m\u00b2)',
                           yaxis = 'y2'
                           #text = df.column (this is the info displayed on hover)
                           )

    # update the data list
    data_plot = [Current_Density]

    # configure the layout
    layout = go.Layout(
                title = 'Current Density',
                #to define axis, can either use go.layout.etc or dict(etc)
                xaxis = go.layout.XAxis(
    #                                     title = 'Date',
                                        tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                        hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                        showgrid = False,
                                        #range=[100,steady_state['Duration(hr)'].max()]
                                        ),    
                yaxis= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
    #                                  overlaying='y',
                                     side = 'right',
                                     showgrid = False,
                                     range = [0,6],
                                     rangemode='tozero'),
                legend= dict(xanchor="right",x=1.2,yanchor="top", y =1.2, bgcolor = 'rgba(0,0,0,0)'),
                template = 'simple_white',
                font = dict(size = 18)
                )


    # create the figure
    fig0 = go.Figure(data = data_plot, layout = layout)
    fig0.update_xaxes(tickangle=90)

    fig0.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                     )


    fig0.write_image('{}/Current_Density.png'.format(image_folder_name), engine="orca")
    fig0.write_html('{}/Current_Density.html'.format(image_folder_name), include_plotlyjs=('cdn'))

    # fig0.show()
    #creating Avg_voltage_chart
    Avg_StackU1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Avg_V_C1_1-C4_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U1 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Avg_StackU2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Avg_V_C5_1-C8_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Avg_StackL2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Avg_V_C9_1-C12_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Avg_StackL1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Avg_V_C13_1-C16_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L1 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Current_Density = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['CURRENT DENSITY'],
                           mode = 'markers',
    #                        mode = 'lines',
                           name = 'Current Density (A/c'+'m\u00b2)',
                           yaxis = 'y2'
                           #text = df.column (this is the info displayed on hover)
                           )

    # update the data list
    data = [
        Avg_StackU1,
        Avg_StackU2,
        Avg_StackL2,
        Avg_StackL1,
        Current_Density]

    # configure the layout
    layout = go.Layout(
                title = 'LHC Avg Cell Voltage'+'<br>'+ '& Current Density',
                #to define axis, can either use go.layout.etc or dict(etc)
                xaxis = go.layout.XAxis(
    #                                     title = 'Date',
                                        tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                        hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                        showgrid = False,
                                        #range=[100,steady_state['Duration(hr)'].max()]
                                        ),    
                yaxis = dict(title='Avg Voltage (V)',
    #                                 font = 15,
                                    showgrid = False,
                                    range=[1.3, 2.3],
                                    rangemode='tozero'),
                yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                     overlaying='y',
                                     side = 'right',
                                     showgrid = False,
                                     range = [0,2],
                                     rangemode='tozero'),
                legend= dict(xanchor="right",x=1.5,yanchor="top", y =2, bgcolor = 'rgba(0,0,0,0)'),
                template = 'simple_white',
                font = dict(size = 18)
                )


    # create the figure
    fig5 = go.Figure(data = data, layout = layout)
    fig5.update_xaxes(tickangle=90)

    fig5.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                     )


    fig5.write_image('{}/LHC_Avg_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), engine="orca")
    fig5.write_html('{}/LHC_Avg_Stack_Voltage_&_Current_Density.html'.format(image_folder_name), include_plotlyjs=('cdn'))
    # fig5.show()

    # creating Max_Voltage & Current Density Chart
    Max_StackU1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Max_V_C1_1-C4_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U1 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Max_StackU2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Max_V_C5_1-C8_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Max_StackL2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Max_V_C9_1-C12_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Max_StackL1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Max_V_C13_1-C16_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L1 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Current_Density = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['CURRENT DENSITY'],
                           mode = 'markers',
    #                        mode = 'lines',
                           name = 'Current Density (A/c'+'m\u00b2)',
                           yaxis = 'y2'
                           #text = df.column (this is the info displayed on hover)
                           )

    # update the data list
    data = [
        Max_StackU1,
        Max_StackU2,
        Max_StackL2,
        Max_StackL1,
        Current_Density]

    # configure the layout
    layout = go.Layout(
                title = 'LHC Max Cell Voltage'+'<br>'+'& Current Density',
                #to define axis, can either use go.layout.etc or dict(etc)
                xaxis = go.layout.XAxis(
    #                                     title = 'Date',
                                        tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                        hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                        showgrid = False,
                                        #range=[100,steady_state['Duration(hr)'].max()]
                                        ),    
                yaxis = dict(title='Max Voltage (V)',
    #                                 font = 15,
                                    range=[1.3, 2.3],
                                    showgrid = False,
                                    rangemode='tozero'),
                yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                     overlaying='y',
                                     range = [0,2],
                                     side = 'right',
                                     showgrid = False,
                                     rangemode='tozero'),#,
    #                                  range=[0,600]),
                legend= dict(xanchor="right",x=1.5,yanchor="top", y =2, bgcolor = 'rgba(0,0,0,0)'),
                template = 'simple_white',
                font = dict(size = 18)
                )

    # create the figure
    fig6 = go.Figure(data = data, layout = layout)
    fig6.update_xaxes(tickangle=90)

    fig6.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                     )

    fig6.write_image('{}/LHC_Max_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), engine="orca")
    fig6.write_html('{}/LHC_Max_Stack_Voltage_&_Current_Density.html'.format(image_folder_name), include_plotlyjs=('cdn'))

    # fig6.show()

    # create min voltage and cd chart
    Min_StackU1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Min_V_C1_1-C4_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Min_StackU2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Min_V_C5_1-C8_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Min_StackL2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Min_V_C9_1-C12_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    Min_StackL1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['Min_V_C13_1-C16_16'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L1 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )   #text = df.column (this is the info displayed on hover)
    Current_Density = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['CURRENT DENSITY'],
                           mode = 'markers',
    #                        mode = 'lines',
                           name = 'Current Density (A/c'+'m\u00b2)',
                           yaxis = 'y2'
                           #text = df.column (this is the info displayed on hover)
                           )                       

    # update the data list
    data = [
        Min_StackU1,
        Min_StackU2, 
        Min_StackL2,
        Min_StackL1,
        Current_Density]

    # configure the layout
    layout = go.Layout(
                title = 'LHC Min Cell Voltage'+'<br>'+'& Current Density',
                #to define axis, can either use go.layout.etc or dict(etc)
                xaxis = go.layout.XAxis(
    #                                     title = 'Date',
                                        tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                        hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                        showgrid = False,
                                        #range=[100,steady_state['Duration(hr)'].max()]
                                        ),    
                yaxis = dict(title='Min Voltage (V)',
    #                                 font = 15,
                                    range = [1.3, 2.3],
                                    showgrid = False,
                                    rangemode='tozero'),
                yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                     range = [0,2],
                                     overlaying='y',
                                     side = 'right',
                                     showgrid = False,
                                     rangemode='tozero'),#,
    #                                  range=[0,600]),
                legend= dict(xanchor="right",x=1.5,yanchor="top", y =2, bgcolor = 'rgba(0,0,0,0)'),
                template = 'simple_white',
                font = dict(size = 18)
                )

    # create the figure
    fig7 = go.Figure(data = data, layout = layout)
    fig7.update_xaxes(tickangle=90)

    fig7.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                     )

    fig7.write_image('{}/LHC_Min_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), engine="orca")
    fig7.write_html('{}/LHC_Min_Stack_Voltage_&_Current_Density.html'.format(image_folder_name), include_plotlyjs=('cdn'))
    # fig7.show()

    # create ASR Analysis Chart
    ASR_StackU1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['ASR_Stack1'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U1 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    ASR_StackU2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['ASR_Stack2'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    ASR_StackL2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['ASR_Stack3'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    ASR_StackL1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['ASR_Stack4'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L1 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )

    # # update the data list
    data = [
        ASR_StackU1,
        ASR_StackU2,
        ASR_StackL2,
        ASR_StackL1, 
        Current_Density]

    # configure the layout
    layout = go.Layout(
                title = 'Stack ASR Chart',
                #to define axis, can either use go.layout.etc or dict(etc)
                xaxis = go.layout.XAxis(
    #                                     title = 'Date',
                                        tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                        hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                        showgrid = False,
                                        #range=[100,steady_state['Duration(hr)'].max()]
                                        ),    
                yaxis = dict(title='ASR (mΩ-c'+'m\u00b2)',
    #                                 font = 15,
                                    range = [0,15000],
                                    showgrid = False,
                                    rangemode='tozero'),
                yaxis2= dict(title='Current Density'+'<br>'+'(A/c'+'m\u00b2)',
                                     overlaying='y',
                                     side = 'right',
                                     showgrid = False,
                                     rangemode='tozero',
                                     range=[0,1.5]),
                legend= dict(xanchor="right",x=1.5,yanchor="top", y=2, bgcolor = 'rgba(0,0,0,0)'),
                template = 'simple_white',
                font = dict(size = 18)
                )

    # create the figure
    fig8 = go.Figure(data = data, layout = layout)

    fig8.update_xaxes(tickangle=90)

    fig8.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                     ) 

    fig8.write_image('{}/Stack_ASR_Chart.png'.format(image_folder_name), engine="orca")
    fig8.write_html('{}/Stack_ASR_Chart.html'.format(image_folder_name), include_plotlyjs=('cdn'))

    # fig8.show()

    #create Max ASR chart
    max_ASR_StackU1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['ASR(max)_Stack1'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U1 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    max_ASR_StackU2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['ASR(max)_Stack2'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack U2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    max_ASR_StackL2 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['ASR(max)_Stack3'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L2 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )
    max_ASR_StackL1 = go.Scattergl(
                           x = Vcell_data['Date'],
                           y = Vcell_data['ASR(max)_Stack4'],
                           mode = 'markers',
                           #mode = 'lines',
                           name = 'Stack L1 (V)',
                           yaxis = 'y1'
                           #text = df.column (this is the info displayed on hover)
                           )

    # update the data list
    data = [
        max_ASR_StackU1,
        max_ASR_StackU2,
        max_ASR_StackL2,
        max_ASR_StackL1,
        Current_Density]

    # configure the layout
    layout = go.Layout(
                title = 'Stack ASR (max) Chart',
                #to define axis, can either use go.layout.etc or dict(etc)
                xaxis = go.layout.XAxis(
    #                                     title = 'Date',
                                        tickformat = "%d/%m/%y"+'<br>'+"  %H:%M",
                                        hoverformat = "%d/%m/%y"+'<br>'+"  %H:%M:%S",
                                        showgrid = False,
                                        #range=[100,steady_state['Duration(hr)'].max()]
                                        ),    
                yaxis = dict(title='Max ASR (mΩ-c'+'m\u00b2)',
                                    range = [0,15000],
    #                                 font = 15,
                                    showgrid = False,
                                    rangemode='tozero'),
                yaxis2= dict(title='Current Density'+'<br>'+ '(A/cm\u00b2)',
                                    range = [0,1.5],
                                     overlaying='y',
                                     side = 'right',
                                     showgrid = False,
                                     rangemode='tozero'),
    #                                  range=[0,600]),
                legend= dict(xanchor="right",x=1.5,yanchor="top", y =2, bgcolor = 'rgba(0,0,0,0)'),
                template = 'simple_white',
                font = dict(size = 18)
                )

    # create the figure
    fig9 = go.Figure(data = data, layout = layout)

    fig9.update_xaxes(tickangle=90)

    fig9.update_layout(xaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)),
           yaxis2 = dict(titlefont = dict(size = 18), tickfont = dict(size = 18)) 
                     ) 

    fig9.write_image('{}/Stack_ASR(max)_Chart.png'.format(image_folder_name), engine="orca")
    fig9.write_html('{}/Stack_ASR(max)_Chart.html'.format(image_folder_name), include_plotlyjs=('cdn'))

    # fig9.show()
    
    prs = Presentation('{}.pptx'.format(template_ppt_fn))

    slide = prs.slides[3]
    height = Inches(3.32)
    width = Inches(5.1)
    row1_y = Inches(0.59)
    row2_y = Inches(4.18)
    column1_x = Inches(0.48)
    column2_x = Inches(7.15)

    # place the LHC_Avg_Stack_Voltage_chart
    pic = slide.shapes.add_picture('{}/LHC_Avg_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), column2_x, row1_y,width=width, height=height)
    pic.click_action.hyperlink.address = '{}/LHC_Avg_Stack_Voltage_&_Current_Density.html'.format(image_folder_name)

    # place the LHC_Max_Stack_Voltage chart
    pic = slide.shapes.add_picture('{}/LHC_Max_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), column1_x, row1_y,width=width, height=height)
    pic.click_action.hyperlink.address = '{}/LHC_Max_Stack_Voltage_&_Current_Density.html'.format(image_folder_name)

    # place the LHC_Min_Stack_Voltage chart
    pic = slide.shapes.add_picture('{}/LHC_Min_Stack_Voltage_&_Current_Density.png'.format(image_folder_name), column1_x, row2_y,width=width, height=height)
    pic.click_action.hyperlink.address = '{}/LHC_Min_Stack_Voltage_&_Current_Density.html'.format(image_folder_name)

#     # place the Current Density Chart
#     pic = slide.shapes.add_picture('{}/Current_Density.png'.format(image_folder_name), column2_x, row2_y,width=width, height=height)
#     pic.click_action.hyperlink.address = '{}/Current_Density.html'.format(image_folder_name)

    prs.save('{}/{}.pptx'.format(output_dir, ppt_fn))

In [62]:
def rungui():

    # Create an instance of tkinter frame
    win = Tk()
    win.title('Lotus Sys Data Visualization')

    # create the frame layout for the main window
    frame = tk.Frame(master=win, width=500, height=175)
    frame.pack()

    # add a label for the automatic data
    label1 = tk.Label(master=frame, text="Data files:")
    label1.place(x=65, y=25)

    # add an entry box for the automatic data file
    entry1 = tk.Entry(win, bg="white", width = 45)
    entry1.place(x=125, y=25)

    # this function opens a file browser for excel files and outputs the selected box to the automatic data entry field
    def browse1():
        try:
            entry1.delete(0, tk.END)
            files = filedialog.askopenfilenames(filetypes=[('CSV Files', '*.csv')])
            for i in files[:-1]:
                entry1.insert(tk.END, '{}, '.format(i))
            entry1.insert(tk.END, files[-1])
        except PermissionError:
            messagebox.showerror('Permission Error', 'Error: Please close the file on your computer and try again.')

    # Create a browse button that links to the browse1 function
    button1 = ttk.Button(win, text="Browse", command=browse1)
    button1.place(x=400, y=21)

    # create a label for the manual data
    label2 = tk.Label(master=frame, text="Stack Information file:")
    label2.place(x=0, y=50)

    # create an entry box for the manual file name
    entry2 = tk.Entry(win, bg="white", width = 45)
    entry2.place(x=125, y=50)

    # this function opens a file browser for excel files and outputs the 
    # selected box to the manual data entry field
    def browse2():
        try:
            file = filedialog.askopenfile(mode='r', filetypes=[('Excel Files', 
                                                                '*.xlsx')])
            filename = os.path.abspath(file.name)
            entry2.insert(tk.END, filename)
        except PermissionError:
            messagebox.showerror('Permission Error', 'Error: Please close the file on your computer and try again.')

    # Create a browse button that links to the browse2 function
    button2 = ttk.Button(win, text="Browse", command=browse2)
    button2.place(x=400, y=47)

    #create a label for the output folder
    label3 = tk.Label(master=frame, text="Output folder:")
    label3.place(x=40, y=77)

    # creat an entry box for the output folder    
    entry3 = tk.Entry(win, bg="white", width = 45)
    entry3.place(x=125, y=77)

    # this function opens a file browser for excel files and outputs the 
    # selected box to the manual data entry field
    def browse3():
        try:
            folder = filedialog.askdirectory()
            #foldername = os.path.abspath(folder.name)
            entry3.insert(tk.END, folder)
        except PermissionError:
            messagebox.showerror('Permission Error', 'Error: Please close the folder on your computer and try again.')

    # Create a browse button that links to the browse3 function
    button3 = ttk.Button(win, text="Browse", command=browse3)
    button3.place(x=400, y=74)
    
    #create a radio item for selecting the template
    label4 = tk.Label(master=frame, text="System name:")
    label4.place(x=0, y=100)
    selected_system = tk.StringVar()
    radio1 = Radiobutton(win, text='U05', value='U05', variable=selected_system)
    radio1.place(x=85, y=100)
    radio2 = Radiobutton(win, text='U0002', value='U0002', variable=selected_system)
    radio2.place(x=140, y=100)
    radio3 = Radiobutton(win, text='SPR0001', value='SPR0001', variable=selected_system)
    radio3.place(x=210, y=100)
    
    # this defines the main run function that we wish to do
    def run_main():
        template_ppt_fn = selected_system.get()
        csv_file_list = entry1.get().split(', ')
        stack_info_fn = entry2.get()
        output_dir = entry3.get()
        main(csv_file_list, stack_info_fn, output_dir, template_ppt_fn)
        win.destroy()
        

    # this is the overall function called that does the main function and the progress bar
    def run_function(name, func):
        # disable the run button
        button4['state'] = 'disabled'
        # define and start the progress bar
        progress_bar = ttk.Progressbar(win, orient = HORIZONTAL, length = 400, mode='indeterminate')
        progress_bar.place(x=50, y = 125)
        progress_bar.start(interval = 15)
        # run the main function
        func()
        # stop the progress bar
        progress_bar.stop()
        # print program complete
        print('Program complete')
        #destory the progress bar
        progress_bar.destroy()
        # show the finished label
        label4 = tk.Label(master=frame, text="Finished!")
        label4.place(x=240, y=115)
        # renable the run button
        button4['state'] = 'enabled'

    # this is the function that threads together the above two functions
    def run_thread(name, func):
        Thread(target = run_function, args=(name,func)).start()

    # this is the function that is called when the run button is clicked
    def run_clicked():
        run_thread('main', run_main)

    # Create a Button that runs the main script
    button4 = ttk.Button(win, text="Run", command=run_clicked)
    button4.place(x=225, y=145)

    # create a help menu

    win.mainloop()
    
rungui()    

Output Rreport name will be:
 2023-June-30_1342hrs_IST_U05_update
['C:/Users/nidhi.dubey/Documents/Lotus_system_data/extracted_files_U05/DATA_U5_2023_05_18.csv', 'C:/Users/nidhi.dubey/Documents/Lotus_system_data/extracted_files_U05/DATA_U5_2023_05_19.csv']
Total Run Duration is "43 hours" (excluding interruptions)
From : 2023-05-18 00:07:15
To   : 2023-05-19 22:23:30
